In [ ]:
import os
os.environ['THEANORC'] = 'theanorc_cpu'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

import torch
import torch.nn as nn
import theano
import numpy as np
import theano 
import theano.tensor as T

print (theano.__version__)

In [ ]:
from ops import pytorch_wrapper
import sys
sys.path.append("./eval_gen/")
torch.backends.cudnn.benchmark = True

# Load data

In [ ]:
import h5py
f = h5py.File('/sdh/data_fuel/mnist.hdf5')

num_test = 24
num_samples= 16

X = f['features'].value[60000:]

permutation = np.random.RandomState(seed=2919).permutation(X.shape[0])
X= X[permutation][:num_test]

# Load model

In [ ]:
dtype= torch.cuda.FloatTensor
age_model = torch.load('netG_epoch_25.pth')
age_model.eval()
age_model.type(dtype)
net = age_model

class NetWrapper(nn.Module):
    '''
        The AIS code needs samples to be of shape (B x 32^2)
        But my network produces (B x 32 x 32)
        So I use this wrapper.
    '''
    def __init__(self, net):
        super(NetWrapper, self).__init__()
        self.net = net
        
    def forward(self, x):
        return self.net(None, x.view(-1,10,1,1)).view(-1,32*32) / 2. + 0.5

# Go

In [ ]:
def generator(z):
    '''
        This function should define theano computational graph
        for evaluating net(z), for `z` -- latent vector of shape [B x Z_dim]
    '''
    
    f = pytorch_wrapper(NetWrapper(net), dtype=dtype)
    out = f(z)
    
    return out


from sampling import samplers_32 as samplers
lld, pf, finalstate = samplers.run_ais(generator, 
                                      X, 
                                      num_samples,
                                      num_steps=10000, 
                                      sigma=0.03, 
                                      hdim=10, 
                                      L=10, 
                                      epsilon=0.01, 
                                      data='continuous',
                                      prior="normal")